## Window Functions
- Perform an operation across a set of rows that are somehow related to the current row.
- Similar to GROUP BY aggregate functions, but all rows remain in the output.
### Uses
- Fetching values from preceding or following rows (e.g. fetching the previous row's value)
1. Determining reigning champion status
2. Calculating growth over time
- Assigning ordinal ranks (1st, 2nd, etc.) to rows based on their values' positions in a sorted list
- Running totals, moving averages.

### ROW_NUMBER() OVER() AS "alias"
Adds a column with each row's number or index.

## ORDER BY in Window Functions
- ORDER BY in OVER orders the rows related to the current row.

## LAG(column, n) OVER(...)
Returns column's value at the row "n" rows before the current row.

In [4]:
WITH Athlete_Medals AS (
  SELECT
    Athlete,
    COUNT(*) AS Medals
  FROM medals.Summer_Medals
  GROUP BY Athlete)

SELECT
  ROW_NUMBER() OVER(ORDER BY medals DESC) AS rank,
  athlete,
  medals
FROM Athlete_Medals
ORDER BY Medals DESC
LIMIT 5;

,rank,athlete,medals
0,1,PHELPS Michael,22
1,2,LATYNINA Larisa,18
2,3,ANDRIANOV Nikolay,15
3,4,MANGIAROTTI Edoardo,13
4,5,ONO Takashi,13


## PARTITION BY
- Splits the table into partitions based on the column's unique values. The results aren't rolled into one column
- Operated on separately by the window function. ROW_NUMBER will reset for each partition, LAG will only fetch a row's previous value if its previous row is in the same partition.

# Relative
### LAG(column, n)
Returns column's value at the row n rows before the current row.
### LEAD(column, n)
Returns column's value at the row n rows after the current row.
# Absolute
### FIRST_VALUE(column)
Returns the first value in the table or partition
### LAST_VALUE(column)
Returns the last value in the table or partition

In [3]:
-- Query that tells us what city hosted the olympics by year.
WITH hosts AS (
    SELECT 
        DISTINCT year,
        city
    FROM medals.summer_medals)
/* Main query that shows the next city that hosts within a given record. */
SELECT
    year,
    city,
    LEAD(city, 1) OVER(ORDER BY year ASC) AS next_city,
    LEAD(city, 2) OVER(ORDER BY year ASC) AS after_next_city
FROM hosts
ORDER BY year ASC;

,year,city,next_city,after_next_city
0,1896,Athens,Paris,St Louis
1,1900,Paris,St Louis,London
2,1904,St Louis,London,Stockholm
3,1908,London,Stockholm,Antwerp
4,1912,Stockholm,Antwerp,Paris
5,1920,Antwerp,Paris,Amsterdam
6,1924,Paris,Amsterdam,Los Angeles
7,1928,Amsterdam,Los Angeles,Berlin
8,1932,Los Angeles,Berlin,London
9,1936,Berlin,London,Helsinki


# Ranking Functions
## ROW_NUMBER()
always assigns unique numbers, even if two rows' values are the same.
## RANK()
Assigns the same number to rows with identical values, skipping over the next numbers in such cases.
## DENSE_RANK()
Also assigns the same number to rows with identical values, but doesn't skip over the next numbers.

In [7]:
-- Query returns the amount of games select countries have participated.
WITH Country_Games AS (
    SELECT
        Country,
        COUNT(DISTINCT year) AS Games
    FROM medals.Summer_Medals
    WHERE
        Country IN ('GBR', 'DEN', 'FRA',
                    'ITA', 'AUT', 'BEL',
                    'NOR', 'POL', 'ESP')
    GROUP BY Country
    ORDER BY Games DESC)
    
/* Ranks each select country by the amount of games played. */
SELECT
    Country,
    Games,
    ROW_NUMBER() OVER(ORDER BY Games DESC) AS ROW_N, --Gives unique number, even if value repeats.
    RANK() OVER(ORDER BY Games DESC) AS Rank_N, --Gives rank number to values, and skips numbers if two values are the same.
    DENSE_RANK() OVER(ORDER BY Games DESC) AS Dense_Rank_N --Similar to RANK() but doesn't skip numbers.
FROM Country_Games
ORDER BY 
    Games DESC,
    Country ASC;

,country,games,row_n,rank_n,dense_rank_n
0,GBR,27,1,1,1
1,DEN,26,3,2,2
2,FRA,26,2,2,2
3,ITA,25,4,4,3
4,AUT,24,5,5,4
5,BEL,24,6,5,4
6,NOR,22,7,7,5
7,POL,20,8,8,6
8,ESP,18,9,9,7


In [12]:
--Query returns each countries athletes, with the amount of medals each athlete won.   
WITH Country_Medals AS (
    SELECT
        Country,
        Athlete,
        COUNT(*) AS Medals
    FROM medals.Summer_Medals
    WHERE
        Country IN('CHN', 'RUS')
        AND Year = 2012
    GROUP BY Country, Athlete
    HAVING COUNT(*) > 1
    ORDER BY Country ASC, Medals DESC)
-- Main Query where Athletes are ranked by the amount of medals earned. Partitioned by the country the athlete was from.
SELECT
    Country,
    Athlete,
    Medals,
    DENSE_RANK() OVER(PARTITION BY Country
                      ORDER BY Medals DESC) AS RANK_N
FROM Country_Medals
ORDER BY Country ASC, Medals DESC;

,country,athlete,medals,rank_n
0,CHN,SUN Yang,4,1
1,CHN,GUO Shuang,3,2
2,CHN,WANG Hao,3,2
3,CHN,ZOU Kai,3,2
4,CHN,HE Zi,2,3
5,CHN,HUANG Xuechen,2,3
6,CHN,LIU Ou,2,3
7,CHN,LI Xiaoxia,2,3
8,CHN,QIN Kai,2,3
9,CHN,SUN Yujie,2,3


### Paging
Splitting data into (approximately equal chunks)
- A use: Many APIs return data in "pages" to reduce data being sent
- Separating data into quartiles or thirds (top middle 33%, and bottom thirds) to judge performance.

## NTILE(n)
Splits the data into n approximately equal pages.

In [15]:
-- CTE Query returns all unique disciplines.
WITH Disciplines AS (
    SELECT
        DISTINCT Discipline
    FROM Medals.Summer_Medals)
-- Query where the data are split by 15 pages.
SELECT
    Discipline,
    NTILE(15) OVER() AS Page
FROM Disciplines
ORDER BY Page ASC;

,discipline,page
0,Wrestling Freestyle,1
1,Archery,1
2,Baseball,1
3,Lacrosse,1
4,Judo,1
...,...,...
62,Beach Volleyball,14
63,Eventing,15
64,Diving,15
65,Triathlon,15


In [18]:
-- CTE Query where each countries medals are counted.
WITH Country_Medals AS (
    SELECT
        Country,
        COUNT(*) AS Medals
    FROM medals.Summer_Medals
    GROUP BY Country),

--query where the Country medals CTE is split into 3 pages.
Thirds AS (
    SELECT
        Country,
        Medals,
        NTILE(3) OVER(ORDER BY Medals DESC) AS Third
    FROM Country_Medals)
    
    -- Query where the third split medals are arranged to show the average amount of medals earned.
SELECT
    Third,
    ROUND(AVG(Medals), 2) AS Avg_Medals
FROM Thirds
GROUP BY Third
ORDER BY Third ASC

,third,avg_medals
0,1,598.74
1,2,22.98
2,3,2.08


In [5]:
-- Query on the amount of medals Brazil won gold from 1992 and beyond.
WITH Brazil_Medals AS (SELECT
        Year,
        Count(*) AS Medals
    FROM medals.Summer_Medals
    WHERE
        Country = 'BRA'
        AND Medal = 'Gold'
        AND Year >= 1992
    GROUP BY Year
    ORDER BY YEAR ASC)
--Query of gold medals that Brazil won each year, with a running total.
SELECT
    Year,
    Medals,
    SUM(Medals) OVER (ORDER BY Year ASC) AS Medals_RT
FROM Brazil_Medals

,year,medals,medals_rt
0,1992,13,13
1,1996,5,18
2,2004,18,36
3,2008,14,50
4,2012,14,64


In [15]:
WITH m AS (SELECT -- Query that produces the amount of gold won by countries in each year.
        year,
        country,
        COUNT(medal) AS g_medal
    FROM medals.summer_medals
    WHERE
        year >= 1992
        AND medal = 'Gold'
    GROUP BY
        year, country
    ORDER BY year ASC, country ASC)
    
SELECT -- Query that gives the running total of gold medals won from 1992 by each country.
    year,
    country,
    g_medal,
    SUM(g_medal) OVER(PARTITION BY country ORDER BY year ASC) AS running_total
FROM m
ORDER BY country ASC, year ASC;


,year,country,g_medal,running_total
0,2012,,2,2
1,1992,ALG,1,1
2,1996,ALG,2,3
3,2000,ALG,1,4
4,2012,ALG,1,5
...,...,...,...,...
302,2012,VEN,1,1
303,1996,YUG,1,1
304,2000,YUG,12,13
305,2004,ZIM,1,1


# ROWS BETWEEN
## Defined by ROWS BETWEEN 'START' AND 'FINISH'
### n PRECEDING
n rows before the current row
### CURRENT ROW
Current Row
### n FOLLOWING
n rows after the current row

In [20]:
WITH Russia_Medals AS (SELECT -- Query of gold medals won by russia by year.
        Year,
        COUNT(*) AS Medals
    FROM medals.Summer_Medals
    WHERE
        Country = 'RUS'
        AND Medal = 'Gold'
    GROUP BY Year
    ORDER BY Year ASC)

SELECT -- Query that gives the max medals and the max medals only from the previous year.
    Year,
    Medals,
    MAX(Medals)
        OVER(ORDER BY Year ASC) AS Max_Medals,
    MAX(Medals)
        OVER(ORDER BY Year ASC
            ROWS BETWEEN
            1 PRECEDING AND CURRENT ROW) AS MAX_Medals_Last
FROM Russia_Medals
ORDER BY Year ASC;

,year,medals,max_medals,max_medals_last
0,1996,36,36,36
1,2000,66,66,66
2,2004,47,66,66
3,2008,43,66,47
4,2012,47,66,47


# Moving average and totals
Moving average (MA): Average of last n periods.

In [24]:
WITH usa AS (SELECT -- Gives the sum of gold medals earned by the us per year
        year,
        COUNT(*) AS medals
    FROM medals.summer_medals
    WHERE
        country = 'USA'
        AND medal = 'Gold'
        AND year >= 1980
    GROUP BY year
    ORDER BY year ASC)

SELECT -- Gives the gold medals won by the US with a 3 year moving average
    year,
    medals,
    ROUND((AVG(medals) OVER
        (ORDER BY year ASC
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)), 2) AS medals_ma
FROM usa
ORDER BY year ASC;

,year,medals,medals_ma
0,1984,168,168.00
1,1988,77,122.50
2,1992,89,111.33
3,1996,160,108.67
4,2000,130,126.33
5,2004,116,135.33
6,2008,125,123.67
7,2012,147,129.33


# ROWS vs RANGE
## RANGE BETWEEN 'START' AND 'FINISH'
- Functions much the same as ROWS BETWEEN
- RANGE treats duplicates in the OVER's ORDER BY subclause as a single entity.

# Pivoting
- Transforms the table by flipping the columns with the record in the data field.
- Easier to scan, especially if pivoted by a chronologically ordered column.
## CROSSTAB
- Must enter: CREATE EXTENSION IF NOT EXISTS tablefunc

In [ ]:
CREATE EXTENSION IF NOT EXISTS tablefunc;

SELECT * FROM CROSSTAB($$
  WITH Country_Awards AS (
    SELECT
      Country,
      Year,
      COUNT(*) AS Awards
    FROM medals.Summer_Medals
    WHERE
      Country IN ('FRA', 'GBR', 'GER')
      AND Year IN (2004, 2008, 2012)
      AND Medal = 'Gold'
    GROUP BY Country, Year)

  SELECT
    Country,
    Year,
    RANK() OVER
      (PARTITION BY Year
       ORDER BY Awards DESC) :: INTEGER AS rank
  FROM Country_Awards
  ORDER BY Country ASC, Year ASC;
-- Fill in the correct column names for the pivoted table
$$) AS ct (Country VARCHAR,
           "2004" INTEGER,
           "2008" INTEGER,
           "2012" INTEGER)

Order by Country ASC;

Error: CREATE EXTENSION IF NOT EXISTS tablefunc;

SELECT * FROM CROSSTAB($$
  WITH Country_Awards AS (
    SELECT
      Country,
      Year,
      COUNT(*) AS Awards
    FROM medals.Summer_Medals
    WHERE
      Country IN ('FRA', 'GBR', 'GER')
      AND Year IN (2004, 2008, 2012)
      AND Medal = 'Gold'
    GROUP BY Country, Year)

  SELECT
    Country,
    Year,
    RANK() OVER
      (PARTITION BY Year
       ORDER BY Awards DESC) :: INTEGER AS rank
  FROM Country_Awards
  ORDER BY Country ASC, Year ASC;
-- Fill in the correct column names for the pivoted table
$$) AS ct (Country VARCHAR,
           "2004" INTEGER,
           "2008" INTEGER,
           "2012" INTEGER)

Order by Country ASC; - permission denied to create extension "tablefunc"

# ROLLUP and CUBE
ROLLUP is a GROUP BY subclause that includes extra rows for group-level aggregations.

# CUBE
- is a non-hierarchical ROLLUP
- It generates all possible group-level aggregations

In [3]:
SELECT
    country,
    medal,
    COUNT(*) AS awards
FROM medals.summer_medals
WHERE
    year = 2008 AND country IN('CHN', 'RUS')
GROUP BY ROLLUP(country, medal)
ORDER BY country ASC, medal ASC;

,country,medal,awards
0,CHN,Bronze,57
1,CHN,Gold,74
2,CHN,Silver,53
3,CHN,None,184
4,RUS,Bronze,56
5,RUS,Gold,43
6,RUS,Silver,44
7,RUS,None,143
8,None,None,327


In [4]:
SELECT
    country,
    medal,
    COUNT(*) AS awards
FROM medals.summer_medals
GROUP BY CUBE(country, medal)
ORDER BY country ASC, medal ASC;

,country,medal,awards
0,,Gold,2
1,,Silver,2
2,,None,4
3,AFG,Bronze,2
4,AFG,None,2
...,...,...,...
496,ZZX,None,48
497,None,Bronze,10369
498,None,Gold,10486
499,None,Silver,10310


# COALESCE
- COALESCE() takes a list of values and returns the first non-null value, goaing from lesft to right
- COALESCE(null, null, 1, null, 2) ? 1
- Useful when using SQL operations that return null's: ROLLUP and CUBE, Piviting, and LAG()

In [6]:
SELECT
    COALESCE(country, 'Both countries') AS country,
    COALESCE(medal, 'All medals') AS medal,
    COUNT(*) AS awards
FROM medals.summer_medals
WHERE
    year = 2008 AND country IN('CHN','RUS')
GROUP BY ROLLUP (country, medal)
ORDER BY country ASC, medal DESC;

,country,medal,awards
0,Both countries,All medals,327
1,CHN,Silver,53
2,CHN,Gold,74
3,CHN,Bronze,57
4,CHN,All medals,184
5,RUS,Silver,44
6,RUS,Gold,43
7,RUS,Bronze,56
8,RUS,All medals,143


# Compressing data
- `STRING_AGG(column, seperator`) takes all the values of a column and concatenates them, with `seperator` in between each value.

In [10]:
WITH Country_Medals AS (
  SELECT
    Country,
    COUNT(*) AS Medals
  FROM medals.Summer_Medals
  WHERE Year = 2000
    AND Medal = 'Gold'
  GROUP BY Country),

  Country_Ranks AS (
  SELECT
    Country,
    RANK() OVER (ORDER BY Medals DESC) AS Rank
  FROM Country_Medals
  ORDER BY Rank ASC)

-- Compress the countries column
SELECT STRING_AGG(Country, ', ')
FROM Country_Ranks
-- Select only the top three ranks
WHERE Rank <= 3;

,string_agg
0,"USA, RUS, AUS"
